### OpenAI Function Calling

The newer OpenAI Function Calling Functionality allows to to define functions which will be passed to the LLM. The LLM
will identify the correct function for the request and provide parameters for the function call.

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
import openai

def chat(query):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
    )
    message = response.choices[0].message.content
    return message

In [ ]:
query = "How much does pizza salami cost?"
message = chat(query)
message

To make use of Function calling you need:

1. A function
2. A dictionary which describes the function

In [ ]:
import json

def get_pizza_info(pizza_name: str):
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return json.dumps(pizza_info)

In [ ]:
functions = [
    {
        "name": "get_pizza_info",
        "description": "Get name and price of a pizza of the restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                "pizza_name": {
                    "type": "string",
                    "description": "The name of the pizza, e.g. Salami",
                },
            },
            "required": ["pizza_name"],
        },
    }
]

In [ ]:
def chat(query):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
        functions=functions, # this is new
    )
    message = response.choices[0].message
    return message

In [ ]:
chat("What is the capital of france?")

In [ ]:
query = "How much does pizza salami cost?"
message = chat(query)
print(message)

In [ ]:
if message.function_call:
    pizza_name = json.loads(message.function_call.arguments).get("pizza_name")
    print(pizza_name)
    function_response = get_pizza_info(
        pizza_name=pizza_name
    )
    print(function_response)

In [ ]:
second_response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": query},
        message,
        {
            "role": "function",
            "name": "get_pizza_info",
            "content": function_response,
        },
    ],
)
second_response

The same can be achieved with LangChain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.openai_functions import create_openai_fn_chain

In [ ]:

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

template = """You are an AI chatbot having a conversation with a human.

Human: {human_input}
AI: """
prompt = PromptTemplate(input_variables=["human_input"], template=template)

chain = create_openai_fn_chain(functions, llm, prompt, verbose=True)

In [ ]:
chain.run("How much does pizza salami cost?")

We can also use Pydantic Classes instead of JSON Schemas

In [ ]:
from pydantic import BaseModel, Field

class GetPizzaInfo(BaseModel):
    """Get name and price of a pizza of the restaurant."""

    pizza_name: str = Field(..., description="The name of the pizza, e.g. Salami")

pydantic_classes = [GetPizzaInfo]

In [ ]:
chain = create_openai_fn_chain(pydantic_classes, llm, prompt, verbose=True)

In [ ]:
chain.run("How much does pizza salami cost?")

We can also pass Functions directly.
To pass Python function in directly, we'll want to make sure our parameters have type hints, we have a docstring, and we use Google Python style docstrings to describe the parameters.

In [ ]:
def get_pizza_info(pizza_name: str) -> dict[str, str]:
    """
    Get name and price of a pizza of the restaurant.

    Args:
        pizza_name: The name of the pizza, e.g. Salami.

    Returns:
        Dict[str, str]: A dictionary containing the name and price of the pizza.
    """
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return pizza_info

In [ ]:
chain = create_openai_fn_chain([get_pizza_info], llm, prompt, verbose=True)

In [ ]:
chain.run("How much does pizza salami cost?")